<a href="https://colab.research.google.com/github/dlsyscourse/lecture14/blob/main/14_hardware_acceleration_architecture_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 14: Hardware Acceleration Implementation

In this lecture, we will to walk through backend scafoldings to get us hardware accelerations for needle.




## Select a GPU runtime type
In this lecture, we are going to make use of c++ and CUDA to build accelerated linear algebra libraries.

In [1]:
!nvidia-smi

Sun Jan  8 02:10:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:25:00.0  On |                  N/A |
| 30%   28C    P8    22W / 350W |     16MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:C1:00.0  On |                  N/A |
| 30%   

## Prepare the codebase

To get started, we can clone the related repo from the github. 

In [ ]:
# # Code to set up the assignment
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/
# !mkdir -p 10714f22
# %cd /content/drive/MyDrive/10714f22
# # comment out the following line if you run it for the second time
# # as you already have a local copy of lecture14
# # !git clone https://github.com/dlsyscourse/lecture14 
# !ln -s /content/drive/MyDrive/10714f22/lecture14 /content/needle

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/10714f22
Cloning into 'lecture14'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 53 (delta 15), reused 50 (delta 12), pack-reused 0
Unpacking objects: 100% (53/53), done.


In [ ]:
# !python3 -m pip install pybind11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 213 kB 5.2 MB/s 


### Build the needle cuda library

We leverage pybind to build a c++/cuda library for acceleration. You can type make to build the corresponding library.

In [1]:
# %cd python/needle
!make clean
!make

rm -rf build python/needle/backend_ndarray/ndarray_backend*.so
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Python: /usr/local/miniconda3/bin/python3.9 (found version "3.9.7") found components: Development Interpreter 
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- Found pybind11: /home/hujunhao/.conda/envs/dlsys/lib/python3.8/site-packages/pybind11/include (found version "2.10.1")
-- Looking for pthread.h
-- Looking

We can then run the following command to make the path to the package available in colab's environment as well as the PYTHONPATH.

In [2]:
# %set_env PYTHONPATH /content/needle/python:/env/python/
import sys
sys.path.append("./python")

## Codebase walkthrough


Now click the files panel on the left side. You should be able to see these files

Python:
- needle/backend_ndarray/ndarray.py
- needle/backend_ndarray/ndarray_backend_numpy.py

C++/CUDA
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu

The main goal of this lecture is to create an accelerated ndarray library.
As a result, we do not need to deal with needle.Tensor for now and will focus on backend_ndarray's implementation. 

After we build up this array library, we can then use it to power backend array computations in needle.


## Creating a CUDA NDArray






In [3]:
from needle import backend_ndarray as nd

ImportError: cannot import name 'ndarray_backend_cpu' from partially initialized module 'needle.backend_ndarray' (most likely due to a circular import) (/home/hujunhao/dlsys/hw3/./python/needle/backend_ndarray/__init__.py)

In [5]:
from needle.backend_ndarry import ndarray_backend_cuda

ImportError: cannot import name 'ndarray_backend_cpu' from partially initialized module 'needle.backend_ndarray' (most likely due to a circular import) (/home/hujunhao/dlsys/hw3/./python/needle/backend_ndarray/__init__.py)

In [ ]:
x = nd.NDArray([1, 2, 3], device=nd.cpu())

In [ ]:
x = nd.NDArray([1,2,3], device=nd.cuda())

In [ ]:
y = x + 1

In [ ]:
y

In [ ]:
y = x + x

In [ ]:
y

NDArray([2. 4. 6.], device=cuda())

We can create a CUDA tensor from the data by specifying a device keyword.

In [ ]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())

In [ ]:
y = x + 1

In [ ]:
x.numpy()

In [ ]:
x.device

cuda()

In [ ]:
y = x + 1

In [ ]:
y.device

cuda()

In [ ]:
y.numpy()

array([2., 3., 4.], dtype=float32)

### Key Data Structures

Key data structures in backend_ndarray

- NDArray: the container to hold device specific ndarray
- BackendDevice: backend device
    - mod holds the module implementation that implements all functions
    - checkout ndarray_backend_numpy.py for a python-side reference.



## Trace GPU execution

Now, let us take a look at what happens when we execute the following code


In [ ]:
x = nd.NDArray([1, 2, 3], device=nd.cuda())
y = x + 1

In [ ]:
x.device.from_numpy

<function needle.backend_ndarray.ndarray_backend_cuda.PyCapsule.from_numpy>

In [ ]:
x = nd.NDArray([1, 2, 3])

In [ ]:
x.device.from_numpy

<function needle.backend_ndarray.ndarray_backend_cuda.PyCapsule.from_numpy>

Have the following trace:

backend_ndarray/ndarray.py
- `NDArray.__add__`
- `NDArray.ewise_or_scalar`
- `ndarray_backend_cpu.cc:ScalarAdd`

In [ ]:
y.numpy()

array([2., 3., 4.], dtype=float32)

Have the following trace:

- `NDArray.numpy`
- `ndarray_backend_cpu.cc:to_numpy`

## Guidelines for Reading C++/CUDA related Files

Read
- src/ndarray_backend_cpu.cc
- src/ndarray_backend_cuda.cu


Optional
- CMakeLists.txt: this is used to setup the build and likely you do not need to tweak it.







## NDArray Data Structure

Open up `python/needle/backend_ndarray/ndarray.py`.

An NDArray contains the following fields:
- handle: The backend handle that build a flat array which stores the data.
- shape: The shape of the NDArray
- strides: The strides that shows how do we access multi-dimensional elements
- offset: The offset of the first element.
- device: The backend device that backs the computation






## Transformation as Strided Computation

We can leverage the strides and offset to perform transform/slicing with zero copy.

- Broadcast: insert strides that equals 0
- Tranpose: swap the strides
- Slice: change the offset and shape 

For most of the computations, however, we will call `array.compact()` first to get a contiguous and aligned memory before running the computation. Actually, the more advanced implementations could finish some of the computation without calling the compact function. But taht is too complex and we choose to always call compact before any computation happens.

In [ ]:
import numpy as np
x = nd.NDArray([0, 1,2,3,4,5], device=nd.cpu_numpy())

In [ ]:
x.numpy()

NameError: ignored

In [ ]:
y = nd.NDArray.make(shape=(3, 2, 2), strides=(2, 1, 0), device=x.device, handle=x._handle, offset=0)
y.numpy()

array([[[0., 0.],
        [1., 1.]],

       [[2., 2.],
        [3., 3.]],

       [[4., 4.],
        [5., 5.]]], dtype=float32)

In [ ]:
x = nd.NDArray([1, 2, 3, 4], device=nd.cpu_numpy())

In [ ]:
x.numpy()

array([1., 2., 3., 4.], dtype=float32)

We can use strides and shape manipulation to create different views of the same array.

In [ ]:
y = nd.NDArray.make(shape=(2, 2), strides=(2, 1), device=x.device, handle=x._handle, offset=0)

In [ ]:
y.numpy()

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [ ]:
z = nd.NDArray.make(shape=(2, 1), strides=(2, 1), device=x.device, handle=x._handle, offset=1)

In [ ]:
z.numpy()

array([[2.],
       [4.]], dtype=float32)

## CUDA Acceleration

Now let us open `src/ndarray_cuda_backend.cu` and take a look at current implementation of GPU ops.


## Steps for adding a new operator implementation
- Add an implementation in `ndarray_backend_cuda.cu`, expose via pybind
- Call into the operator in ndarray.py
- Write up testcases

In [ ]:
!make

-- Found pybind11: /usr/local/lib/python3.7/dist-packages/pybind11/include (found version "2.10.0")
-- Found cuda, building cuda backend
Tue Oct 11 04:01:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W /  70W |    104MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+------------------------------

In [ ]:
x = nd.NDArray([1,2,3], device=nd.cuda())

In [ ]:
!python tests/test_mul.py

python3: can't open file 'tests/test_mul.py': [Errno 2] No such file or directory


## Connect back to needle Tensor

So far we only played with the `backend_ndarray` subpackage, which is a self-contained ndarray implementation within needle.

We can connect the ndarray back to needle as a backend.

In [ ]:
import needle as ndl

In [ ]:
x = ndl.Tensor([1,2,3], device=ndl.cpu(), dtype="float32")
y = ndl.Tensor([2,3,5], device=ndl.cpu(), dtype="float32")
z = x + y

In [ ]:
z.device

cpu()

In [ ]:
type(z.cached_data)

needle.backend_ndarray.ndarray.NDArray

## Write Standalone Python Test Files

Now that we have additional c++/cuda libraries in needle, we will need to type make in order to rebuild the library. Additionally, because the colab environment caches the old library, it is inconvenient to use the ipython cells to debug the updated library.




In [ ]:
!make

-- Found pybind11: /usr/local/lib/python3.7/dist-packages/pybind11/include (found version "2.8.1" )
-- Found cuda, building cuda backend
-- Autodetected CUDA architecture(s):  3.7
-- Configuring done
-- Generating done
-- Build files have been written to: /content/drive/MyDrive/10714/lecture17/build
make[1]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[2]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
[  0%] Built target ndarray_backend_cuda
make[3]: Entering directory '/content/drive/MyDrive/10714/lecture17/build'
make[3]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
[ 50%] Built target ndarray_backend_cpu
make[2]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'
make[1]: Leaving directory '/content/drive/MyDrive/10714/lecture17/build'



We recommend writing separate python files and invoke them from the command line. Create a new file `tests/mytest.py` and write your local tests. This is also a common develop practice in big projects that involves python c++ FFI.

In [ ]:
!python tests/mytest.py

python3: can't open file 'tests/mytest.py': [Errno 2] No such file or directory


After we have building the library, we could choose to fully restart the runtime (factory reset runtime) if you want to bring the updated change back to another colab. Note that you will need to save your code changes to the drive or a private github repo.